In [109]:
import numpy as np
import scipy.io
import scipy.stats
import cv2
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from os import listdir

## Demo for SVD

In [3]:
A = [[1, 2, 3], [4, 5, 6], [7, 8, 9]] # matrix for the demo

In [4]:
u, s, v = np.linalg.svd(A) # A = u.s.v'
print("Matrix u is:\n", u, "\n")
print("Matrix s is:\n", s, "\n")
print("Matrix v is:\n", v, "\n")

Matrix u is:
 [[-0.21483724  0.88723069  0.40824829]
 [-0.52058739  0.24964395 -0.81649658]
 [-0.82633754 -0.38794278  0.40824829]] 

Matrix s is:
 [1.68481034e+01 1.06836951e+00 3.33475287e-16] 

Matrix v is:
 [[-0.47967118 -0.57236779 -0.66506441]
 [-0.77669099 -0.07568647  0.62531805]
 [-0.40824829  0.81649658 -0.40824829]] 



### Demo for numpy functions

In [5]:
# numpy.add
x1 = np.arange(9.0)
print("X1 is: \n", x1)

X1 is: 
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]


In [6]:
x2 = x1.reshape(3, 3)
print("X2 is: \n", x2)

X2 is: 
 [[0. 1. 2.]
 [3. 4. 5.]
 [6. 7. 8.]]


In [7]:
x3 = np.arange(3.0)
print("X3 is: \n", x3)

X3 is: 
 [0. 1. 2.]


In [8]:
print("X2 + X3 is:\n", np.add(x2, x3))

X2 + X3 is:
 [[ 0.  2.  4.]
 [ 3.  5.  7.]
 [ 6.  8. 10.]]


In [9]:
x4 = np.array([-1+1j, -1, 0, 3])
print("X4 is:\n", x4)

X4 is:
 [-1.+1.j -1.+0.j  0.+0.j  3.+0.j]


In [10]:
x5 = np.absolute(x4)
print("X5 is:\n", x5)

X5 is:
 [1.41421356 1.         0.         3.        ]


In [11]:
x6 = np.sum(x5)
print("X6 is:\n", x6)

X6 is:
 5.414213562373095


In [12]:
x7 = np.eye(3) + 1j * np.eye(3)
print("X7 is:\n", x7)

X7 is:
 [[1.+1.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+1.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+1.j]]


In [13]:
x8 = np.conjugate(x7)
print("X8 is:\n", x8)

X8 is:
 [[1.-1.j 0.-0.j 0.-0.j]
 [0.-0.j 1.-1.j 0.-0.j]
 [0.-0.j 0.-0.j 1.-1.j]]


### Extracting the feature vectors of the image using its reference image by SVD

In [14]:
# exploiting the vectorization methods already included in the numpy library
def get_feature_vector_2(ref_image, pred_image):
    u_r, s_r, v_r = np.linalg.svd(ref_image[:400, :400])
    u_p, s_p, v_p = np.linalg.svd(pred_image[:400, :400])
    feature_vec = np.add(np.absolute(np.sum(u_r.conj()*u_p, axis=0)), np.absolute(np.sum(v_r.conj()*v_p, axis=0)))
    return feature_vec

### Load the .mat files 

In [31]:
dmos_mat = scipy.io.loadmat('Live Database/databaserelease2/dmos.mat')
ref_names = scipy.io.loadmat('Live Database/databaserelease2/refnames_all.mat')

# extraxt the arrays from the loaded mat files
refnames_all = ref_names['refnames_all']
orgs = dmos_mat['orgs']
dmos = dmos_mat['dmos']

### Load the reference images

In [46]:
ref_images_names = listdir("Live Database/databaserelease2/refimgs/")
# print(ref_images_names)
ref_images = {} # load the images in this dictionary
ref_images_count = len(ref_images_names)
for i in range(ref_images_count):
    ref_images[ref_images_names[i]] = cv2.imread('Live Database/databaserelease2/refimgs/' + str(ref_images_names[i]), 0)
    


### Create the feature vector

In [64]:
X = []
folders = ['jp2k', 'jpeg', 'wn', 'gblur', 'fastfading']
images = [227, 233, 174, 174, 174]
done = 0

In [65]:
for f in range(len(folders)):
    for i in range(images[f]):
        path = 'Live Database/databaserelease2/'+ folders[f] + '/img' + str(i + 1) + '.bmp'
        img_p = cv2.imread(path, 0)
        ref = refnames_all[:, done + i]
        img = ref_images[ref[0][0]]
        feature_vector = get_feature_vector_2(img, img_p)
        X.append(feature_vector)
    done += images[f]

In [68]:
print(ref)
print(ref[0][0])
print(done)

[array(['sailing4.bmp'], dtype='<U12')]
sailing4.bmp
982


In [90]:
X = np.array(X)
Y = dmos[0] # Create the output DMOS matrix for the features

### Split into training and testing data

In [94]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

### Create the SVR classifier, fit on train data, predict on test data

In [105]:
clf = SVR(C = 1.0, epsilon = 0.1)

In [106]:
clf.fit(X_train, Y_train)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [107]:
Y_pred = clf.predict(X_test)

### Compute the value of PLCC

In [110]:
plcc, pval = scipy.stats.pearsonr(Y_test, Y_pred)
print(plcc, pval)

0.9276287906340326 2.4326823632934688e-85


### Dump the results in csv file